# Enhancing predictions of patient conveyance using emergency call handler free text notes for unconscious and fainting incidents reported to the London Ambulance Service

## Preparation

### Load main packages and config

In [ ]:
import copy
from datetime import datetime
import string
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

# ↓Required otherwise display of text strings truncated
pd.set_option("display.max_colwidth", 10000)

### Read in data

In [ ]:
# Read in data set
las18_p31 = pd.read_csv(
    "~/msc-las/data/nlp_data_extract_010118-311218.csv", encoding="ISO-8859–1")

## Data wrangling

### Outcome measures

In [ ]:
# Fill nan values in vehicle dispatched column
las18_p31["vehiclesarrivedx"] = las18_p31["vehiclesarrived"].fillna(0)

# Create binary dispatch attribute for if one or more vehicles was dispatched
las18_p31["dispatched"] = np.where(las18_p31["vehiclesarrivedx"] == 0, 0, 1)

# Create outcome attribute covering the four patient outcomes
las18_p31["conveyed"] = las18_p31["conveyed"].fillna(0)
las18_p31["conveyed_ed"] = las18_p31["conveyed_ed"].fillna(0)
las18_p31["conveyed_other"] = las18_p31["conveyed"] - las18_p31["conveyed_ed"]
las18_p31["outcome_num"] = las18_p31["dispatched"] + \
    las18_p31["conveyed"] + las18_p31["conveyed_ed"]

### Tidy ampds attribute

In [ ]:
# Remove ampds codes incorrectly read in log form
las18_p31["ampdscode"] = las18_p31["ampdscode"].str.replace(
    "3.10E+0", "31E", regex=False)

# Create combined ampds_full description/code attribute for easy reading in charts
las18_p31["ampds_full"] = las18_p31[["ampdscode", "description"]].apply(
    lambda x: " ".join(x), axis=1)

In [ ]:
# Tidying ampds_full descriptions consistent with 
#https://wiki.radioreference.com/index.php/Priority_Dispatch_Codes

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31D2 Unconscious or Fainting - Effective Breathing", 
    "31D2 Unconscious - Effective Breathing", regex=True)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31D3 Unconscious or Fainting - Not Alert", "31D3 Not Alert", regex=True)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31D1 Unconscious  Agonal / Ineffective Breathing", 
    "31D1 Unconscious - Agonal / Ineffective Breathing", regex=True)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31D4 Unconscious or Fainting - Changing Colour",
    "31D4 Changing Colour", regex=True)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31D4 Unconscious or Fainting - Changing Colour", 
    "31D4 Changing Colour", regex=True)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31E2 Unconscious or Fainting - Ineffective Breathing", 
    "31E2 Ineffective Breathing", regex=True)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31C1 Unconscious or Fainting - Alert with Abnormal Breathing", 
    "31C1 Alert with Abnormal Breathing", regex=True)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31C1 Unconscious or Fainting - Alert with Abnormal Breathing", 
    "31C1 Alert with Abnormal Breathing", regex=True)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31E1 Unconscious or Fainting (near) Echo Override", 
    "31E1 Echo Override", regex=False)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31D0 Unconscious or Fainting (near) Delta Override", 
    "31D0 Delta Override", regex=False)

las18_p31["ampds_full"] = las18_p31["ampds_full"].str.replace(
    "31C0 Unconscious or (Near) Fainting Charlie Override", 
    "31C0 Charlie Override", regex=False)

### Process text

In [ ]:
import re
import nltk

In [ ]:
# String length of problem description
las18_p31["string_len"] = las18_p31.problemdescription.str.len()

# Strip all numbers from problem description to remove dates of birht
las18_p31["prob_desc"] = las18_p31.problemdescription.apply(
    lambda x: re.sub(r"[0-9]+", "", str(x)))

# Drop raw problem descrption
las18_p31 = las18_p31.drop("problemdescription", axis=1)

# Lower case the problem description
las18_p31["prob_desc"] = las18_p31["prob_desc"].astype(
    str).apply(lambda x: x.lower())

# New attribute with punctucation removed
las18_p31["prob_desc_no_punc"] = las18_p31.prob_desc.apply(
    lambda x: x.translate(str.maketrans("", "", string.punctuation)))

# String length after removing punctuation and numbers
las18_p31["string_len_p"] = las18_p31["prob_desc_no_punc"].str.len()

In [ ]:
import spacy

In [ ]:
# Lemmatize text

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def lemma(text):
    doc = nlp(text)
    # Extract the lemma for each token and join
    return " ".join([token.lemma_ for token in doc])


las18_p31["prob_desc_lemmat"] = las18_p31["prob_desc"].apply(
    lambda x: lemma(x))

### Save wrangled data frame

In [ ]:
# Create a Pickle with processed data
import os

las18_p31.to_pickle(os.path.join("pickle","las18_p31.pickle"))

del(las18_p31)
las18_p31 = pd.read_pickle(os.path.join("pickle",
                                              "las18_p31.pickle"))

## Split and filter data

### Train-test split

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
# Stratified sample (to preserve ratio of outcomes after dropping
# no-dispatch calls - relevant for complete analysis undertaken in MSc
# Project but not for published paper)

amb_train_0, amb_test_0 = train_test_split(
    las18_p31, test_size=0.20, random_state=100, 
    stratify=las18_p31["outcome_num"])

### Drop if no face-to-face response

In [ ]:
# Keep only instances where ambulance was dispatched
amb_train = amb_train_0.loc[amb_train_0.dispatched != 0]
amb_train = copy.deepcopy(amb_train) # Create as new object

amb_test = amb_test_0.loc[amb_test_0.dispatched != 0]
amb_test = copy.deepcopy(amb_test)

no_disp = len(amb_train_0) - len(amb_train)
str("Number of calls with no dispatch: ") + str(no_disp)

## Summary statistics

In [ ]:
# Function to generate summary table for report

def desc_stats1(vab, df, partition):
    x = df.groupby(by=[vab, "conveyed"]).count()["incidentid"].unstack()
    x["n"] = x.sum(axis=1).astype(int) # Total (conveyed + not conveyed) 
    x["n/N (%)"] = x.n/x.n.sum()*100  # % of total
    x["n/N (%)"] = x["n/N (%)"].round(1)
    x["conveyed/n (%)"] = x[1.0]/x["n"]*100 # Conveyance rate
    x["conveyed/n (%)"] = x["conveyed/n (%)"].round(1)
    x = x.rename(columns={1.0: "conveyed"}) # Number conveyed
    x.index.name = None
    x = x.drop(columns=0.0) # Remove not conveyed number
    x = x[["n", "n/N (%)", "conveyed", 
       "conveyed/n (%)"]].rename_axis("", axis="columns") # Define column order
    x = pd.concat([x], keys=[partition], names=[""], axis=1)
    return x


### MPDS code

In [ ]:
# Number of MPDS P31 determinant codes
len(amb_train.ampds_full.unique())

In [ ]:
desc_stats1("ampds_full", amb_train, "Training set")

### Free text

In [ ]:
# Average token length (excluding punctuation) TRAINING SET

def av_length(df):
    free_text = df.prob_desc.to_string(index=False)
    free_text = nltk.word_tokenize(free_text)

    all_lengths = []
    num_of_strings = len(free_text)

    for item in free_text:
        if item not in string.punctuation:
            string_size = len(item)
            all_lengths.append(string_size)
            total_size = sum(all_lengths)
    ave_size = float(total_size) / float(num_of_strings)
    
    return ave_size, len(set(free_text))

In [ ]:
training_text = {"Average string length": 
                amb_train["string_len"].mean().round(1),
                "Average string length (not conveyed)":
                amb_train.loc[amb_train.conveyed ==0 ]["string_len"].mean().round(1),
               "ave_size_unique": av_length(amb_train)}

pd.DataFrame({"Training set":training_text})

In [ ]:
import wordcloud
from wordcloud import WordCloud, ImageColorGenerator

In [ ]:
# Set an empty list to use as stopwords in Word Cloud (don't want stopwords 
# removed)
stopwords = []

In [ ]:
# All text as one string
all_text = amb_train.prob_desc.to_string(index=False)

# Make Word Cloud
wordcloud = WordCloud(background_color="white",
                      width=1009, height=750, stopwords = stopwords,
                      collocations=False).generate(all_text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
wordcloud.to_file("figures_new/cloud.png")

In [ ]:
# All text as one string

# Make Word Cloud
wordcloud = WordCloud(background_color="white",
                      width=1009, height=750, stopwords = stopwords,
                      collocations=True).generate(all_text)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
wordcloud.to_file("figures_new/cloud2.png")

In [ ]:
# Unigram, bigram and trigram plots excluding punctuation

fig = plt.figure(figsize=(15, 10))
fig.subplots_adjust(wspace=1)

free_text = amb_train.prob_desc_no_punc.to_string(index=False)
free_text = nltk.word_tokenize(free_text)

unigrams = nltk.FreqDist(free_text).most_common(20) # Only top 20
unigrams = list(zip(*unigrams)) # Separate list of pairs in to list of 2 tuples
unigrams_bg = list(unigrams[0])
unigrams_bg.reverse() # Needed to get top bar as highest 
unigrams_freq = list(unigrams[1])
unigrams_freq.reverse() # Needed to get top bar as highest
ax1 = fig.add_subplot(1, 3, 1) # Left plot
ax1.barh(unigrams_bg, unigrams_freq, color="green")
ax1.set_title("Unigrams")

# As above for bigrams
bigrams = nltk.FreqDist(nltk.bigrams(free_text)).most_common(20)
bigrams_bg = [" ".join(pair[0]) for pair in bigrams]
bigrams_bg.reverse()
bigrams_freq = list(list(zip(*bigrams))[1])
bigrams_freq.reverse()
ax2 = fig.add_subplot(1, 3, 2)
ax2.barh(bigrams_bg, bigrams_freq, color="green")
ax2.set_title("Bigrams")

# As above for trigrams
trigrams = nltk.FreqDist(nltk.trigrams(free_text)).most_common(25)
trigrams_bg = [" ".join(pair[0]) for pair in trigrams]
trigrams_bg.reverse()
trigrams_freq = list(list(zip(*trigrams))[1])
trigrams_freq.reverse()
ax3 = fig.add_subplot(1, 3, 3)
ax3.barh(trigrams_bg, trigrams_freq, color="green")
ax3.set_title("Trigrams")
fig.savefig("figures_new/ngrams.jpg")

In [ ]:
amb_train["string_len"].max()

In [ ]:
# X axis labels
amb_train["string_len_band"] = pd.cut(las18_p31.string_len, 
                                      bins=np.arange(0, 250, 10).tolist())

# String length bands frequency and conveyance
age_agg = pd.DataFrame(amb_train.groupby(
    by=["string_len_band", "conveyed"]).count()["incidentid"]).unstack()
age_agg.columns = age_agg.columns.droplevel(0)
age_agg["Incident volume"] = age_agg[0.0] + age_agg[1.0]
age_agg["% conveyed"] = age_agg[1.0]*100/ age_agg["Incident volume"]

bands = pd.IntervalIndex(age_agg.reset_index()["string_len_band"]).mid

fig = plt.figure(figsize=(10, 7))
ax1 = fig.add_subplot(111)

ax1.set_xlabel('string_len')
ax1.grid(False)
ax1.set_ylabel('Incident volume', color="darkgreen")
ax1.bar(bands, age_agg["Incident volume"], color="darkgreen", width=4)
ax1.tick_params(axis='y', labelcolor="darkgreen")
ax1.set_xlim(0,110)
ax1.set_xticks(np.arange(0,240,20))
ax1.set_ylim(0,10000)
ax1.set_yticks(np.arange(0,10000,1000))
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

ax2.set_ylabel('% conveyed', color="darkblue")
ax2.scatter(bands, age_agg["% conveyed"], color="darkblue")
ax2.plot(bands, age_agg["% conveyed"], color="darkblue")
ax2.tick_params(axis='y', labelcolor="darkblue")
ax2.set_ylim(0,100)
ax2.set_yticks(np.arange(0,110,20))

fig.tight_layout()
plt.show()
fig.savefig("figures_new/str_len.jpg")

## Modelling

### Model classifers and functions

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder, FunctionTransformer, LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

from sklearn.cluster import KMeans

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.decomposition import PCA, TruncatedSVD

from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_validate 
from sklearn.model_selection import RandomizedSearchCV, cross_val_predict

In [ ]:
from sklearn.metrics import make_scorer

# Create specificity and true negative scorers
specificity = make_scorer(metrics.recall_score, pos_label=0)
true_negative = make_scorer(metrics.precision_score, pos_label=0)

In [ ]:
# Main classifiers
classifiers1 = {"RF": RandomForestClassifier(n_estimators=100, 
                                             random_state=0, 
                                             min_samples_leaf=50, n_jobs=-1),
                "GBM": GradientBoostingClassifier(
    min_samples_leaf=50, max_depth=100 ), 
                "RF_bal": RandomForestClassifier(n_estimators=100, 
                                                 random_state=0, 
                                                 class_weight="balanced", 
                                                 min_samples_leaf=50, n_jobs=-1)}

# Define label
y_varb = "conveyed"

# Define scoring metrics NB recall = sensitivity
scoring = {"accuracy": "accuracy", "precision": "precision",
           "roc_auc": "roc_auc",
           "sensitivity": "recall", "specificity": specificity,
           "true_negative": true_negative}

In [ ]:
# Function to incorporate pipelines and fitting of models

def process_results_1(key, pipeline, variables1, results, svd):
    # Fit model
    pipeline.fit(amb_train[variables1].values,
                 amb_train[y_varb].values)

    # Feature importances
    if svd:
        feature_names = np.nan
        imps = np.nan
        estimator = np.nan
    else:
        if key == "LSV_bal": # LSV has coefs rather than feature imps
            feature_names = pipeline.named_steps["process"].named_steps["final"].get_feature_names(
            )
            zit = pd.Series(pipeline.named_steps["classifier"].coef_[
                            :, 0], index=feature_names).sort_values(ascending=False)
            imps = zit[0:20].round(2).to_dict()
            estimator = np.nan
        else:
            feature_names = pipeline.named_steps["process"].named_steps["final"].get_feature_names(
            )
            imps = pd.Series(pipeline.named_steps["classifier"].feature_importances_, index=feature_names).sort_values(
                ascending=False)
            imps = imps[0:10].round(2).to_dict()

            # for Decision tree viz
            estimator = pipeline.named_steps["classifier"].estimators_[10]

    return feature_names, imps, estimator

# Function to incorporate cross validation and results processing
def process_results_2(key, pipeline, variables1, variables2, results,
                      model, variant, feature_names, imps, estimator):
    
    # Cross validate model
    output = cross_validate(pipeline, amb_train[variables1].values,
                            amb_train[y_varb].values,
                            scoring=scoring, cv=5, return_train_score=False,
                            return_estimator=True)

    mean = output["test_roc_auc"].mean()

    # Predict on test set
    pipeline.predict(amb_test[variables1].values)
    y_pred = pipeline.predict(amb_test[variables1].values)
    y_pred_proba = pipeline.predict_proba(amb_test[variables1].values)

    # Metrics
    ex_sensitivity = metrics.recall_score(amb_test[y_varb].values, y_pred).round(3)
    ex_specificity = specificity(
        pipeline, amb_test[variables1].values, amb_test[y_varb].values).round(3)

    ex_precision = metrics.precision_score(
        amb_test[y_varb].values, y_pred).round(3)
    ex_true_negative = true_negative(
        pipeline, amb_test[variables1].values, amb_test[y_varb].values).round(3)
    ex_roc_auc = metrics.roc_auc_score(
        amb_test[y_varb].values, y_pred_proba[:, 1]).round(3)

    # IDs of correctly classified
    test_predict_bool = y_pred == 0
    not_conv_id = amb_test["incidentid"].loc[(
        test_predict_bool) & (amb_test.conveyed == 0)].values

    # DeLong CI
    auc, auc_cov = delong_roc_variance(
        amb_test[y_varb].values,
        y_pred_proba[:, 1])

    alpha = 0.95

    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

    ci = stats.norm.ppf(lower_upper_q, loc=auc, scale=auc_std)
    ci[ci > 1] = 1

    roc_auc_ci = f" {ex_roc_auc.round(3)} ( {ci[0].round(3)} - {ci[1].round(3)} )"
    
    # List of dictionaries for results tables
    results.append({"Model": str(model) + str(key), "Classifier": key, "CV sensitivity": output["test_sensitivity"].mean().round(3),
                    "CV specificity": output["test_specificity"].mean().round(3),
                    "CV precision": output["test_precision"].mean().round(3),
                    "CV true negative rate": output["test_true_negative"].mean().round(3),
                    "CV roc auc": mean.round(3),
                    "Feature Imps": imps, "Features": ", ".join(
        variables2), "Variant": variant, "Test precision": ex_precision,
                    "Test sensitivity": ex_sensitivity, "Test specificity": ex_specificity,
                    "Test true negative rate": ex_true_negative,
                    "Test roc auc": ex_roc_auc, "Test roc auc (CI)": roc_auc_ci,
                    "Test y_pred_proba": y_pred_proba, "Test y_pred": y_pred,
                    "Estimator": estimator,  "Feature Names": feature_names,
                    "Test not conveyed IDs": not_conv_id})

### Model run functions

In [ ]:
# Run model categorical variable only
def modelrun_cat(variables, model, variant, classifiers=classifiers1,
                 svd=None, clustering=None):
    results = []  # list to contain outputs from each classifier
    variables1 = variables
    variables2 = variables
    if svd:
        categories_pipe = Pipeline(steps=[("imputer",
                                           SimpleImputer(strategy="constant",
                                                         fill_value="missing")), (
            "final", OneHotEncoder(handle_unknown="ignore")),
                                          ("truncsvd", TruncatedSVD(svd))])

    else:
        categories_pipe = Pipeline(steps=[(
            "imputer", SimpleImputer(
            strategy="constant",
                fill_value="missing")), ("final",
                                           OneHotEncoder(handle_unknown="ignore"))])

    for key, value in classifiers.items():  # for each classifier
        pipeline = Pipeline(
            [("process", categories_pipe,), ("classifier", classifiers[key]), ])

        ret = process_results_1(key, pipeline, variables1, results, svd)
        feature_names = ret[0]
        imps = ret[1]
        estimator = ret[2]

        process_results_2(key, pipeline, variables1, variables2,
                          results, model, variant, feature_names, imps, estimator)
    return pd.DataFrame(results).set_index(["Model", "Features", "Variant", "Classifier"])

In [ ]:
# Run model with count vectorizer on text data
def modelrun_text(text, model, variant, classifiers=classifiers1, svd=None, min_df=10, ngram_range=(1, 1)):
    results = []  # list to contain outputs from each classifier
    variables1 = text
    variables2 = []
    variables2.append(text)
    if svd:
        text_pipe = Pipeline([(
            "final", CountVectorizer(analyzer="word",
                                     token_pattern=r"(?u)\b\w\w+\b|!|\?|\"|\'",
                                                        ngram_range=ngram_range,
                                     min_df=min_df)),
            ("truncsvd", TruncatedSVD(svd))])

    else:
        text_pipe = Pipeline([(
            "final", CountVectorizer(analyzer="word",
                                     token_pattern=r"(?u)\b\w\w+\b|!|\?|\"|\'",
                                                        ngram_range=ngram_range, min_df=min_df))])

    for key, value in classifiers.items():  # for each classifier
        pipeline = Pipeline(
            [("process", text_pipe,), ("classifier", classifiers[key]), ])

        ret = process_results_1(key, pipeline, variables1, results, svd)
        feature_names = ret[0]
        imps = ret[1]
        estimator = ret[2]

        process_results_2(key, pipeline, variables1, variables2,
                          results, model, variant, feature_names, imps, estimator)
    return pd.DataFrame(results).set_index(["Model", "Features", "Variant", "Classifier"])

In [ ]:
# Run model with TFIDF vectorizer on text data
def modelrun_text_tfidf(text, model, variant, classifiers=classifiers1,
                        svd=None, min_df=10, ngram_range=(1, 1)):
    results = []  # list to contain outputs from each classifier
    variables1 = text
    variables2 = []
    variables2.append(text)
    if svd:
        text_pipe = Pipeline([(
            "final", ), ("truncsvd", TruncatedSVD(svd))])

    else:
        text_pipe = Pipeline([("final", TfidfVectorizer(
            analyzer="word", token_pattern=r"(?u)\b\w\w+\b|!|\?|\"|\'", ngram_range=ngram_range, min_df=min_df))])

    for key, value in classifiers.items():  # for each classifier
        pipeline = Pipeline(
            [("process", text_pipe,), ("classifier", classifiers[key]), ])

        ret = process_results_1(key, pipeline, variables1, results, svd)
        feature_names = ret[0]
        imps = ret[1]
        estimator = ret[2]

        process_results_2(key, pipeline, variables1, variables2,
                          results, model, variant, feature_names, imps, estimator)
    return pd.DataFrame(results).set_index(["Model", "Features", "Variant", "Classifier"])

Required to allow extraction of feature names from Pipeline in Column Transformer:
https://stackoverflow.com/questions/48005889/get-features-from-sklearn-feature-union

In [ ]:
class myPipeline(Pipeline):
    def get_feature_names(self):
        for name, step in self.steps:
            if isinstance(step, CountVectorizer):
                return step.get_feature_names()

In [ ]:
# Run model for mixed feature case
def modelrun_all_count(model, variant, classifiers=classifiers1, text=None,
                       others=None, loc_clusters=None):
    variables = []  # list to contain all variables
    results = []  # list to contain outputs from each classifier
    num_vars = {}  # dict to contain numerical variable positions
    cat_vars = {}  # dict to contain cat variables positions
    listy = []  # list to contain all elements of main pipeline

    numerical_pipe = Pipeline(
        steps=[("imputer", SimpleImputer()), ("scale", StandardScaler())])

    text_pipe = myPipeline([("vect", CountVectorizer(
        analyzer="word", token_pattern=r"(?u)\b\w\w+\b|!|\?|\"|\'")), ])

    listy.append(("text", text_pipe, 0))
    variables.append(text)

    variables = variables + others
    for i in range(len(others)):
        if amb_train[others[i]].dtype == "object":
            cat_vars[others[i]] = i + 1
        else:
            num_vars[others[i]] = i + 1

    categories_pipe = myPipeline(steps=[("imputer", SimpleImputer(
        strategy="constant", fill_value="missing")),
                                        ("onehot", 
                                         OneHotEncoder(handle_unknown="ignore"))])

    if cat_vars:  # If there are categorical features
        listy.append(("categories", categories_pipe, [*cat_vars.values()]))

    variables1 = variables # Needed to be consistent with other functions
    variables2 = variables

    for key, value in classifiers.items():  # For each classifier
        pipeline = myPipeline(
            [("union", ColumnTransformer(
                listy, transformer_weights={"text": 1, "num": 1,
                                            "categories": 1, "clust": 1},)),
             ("classifier", classifiers[key]), ])

        # Fit model
        pipeline.fit(amb_train[variables].values,
                     amb_train[y_varb].values)

        feature_names1 = pipeline.named_steps["union"].transformers_[
            0][1].named_steps["vect"].get_feature_names()

        feature_names2 = pipeline.named_steps["union"].transformers_[
            1][1].named_steps["onehot"].get_feature_names()

        feature_names = feature_names1 + feature_names2.tolist()
        feature_names = feature_names + list(num_vars.keys())
        
        imps = pd.Series(pipeline.named_steps["classifier"].feature_importances_,
                         index=feature_names).sort_values(ascending=False)
        imps = imps[0:20].round(2).to_dict()

        estimator = pipeline.named_steps["classifier"].estimators_[90]

        process_results_2(key, pipeline, variables1, variables2,
                          results, model, variant, feature_names, imps, estimator)
    return pd.DataFrame(results).set_index(["Model", 
                                            "Features", "Variant", "Classifier"])

In [ ]:
from scipy import stats

DeLong method code taken from:
    https://github.com/yandexdataschool/roc_comparison/blob/master/compare_auc_delong_xu.py

and
        
https://stackoverflow.com/questions/19124239/scikit-learn-roc-curve-with-confidence-intervals

In [ ]:
def compute_midrank(x):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and (Z[j] == Z[i]).all():
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.
    Args:
       x - a 1D numpy array
    Returns:
       array of midranks
    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=np.float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight=None):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count  # ordered_sample_weight


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    sample_weight = None
    order, label_1_count = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    assert len(
        aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    sample_weight = None
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack(
        (predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)

### Run the models

In [ ]:
results = modelrun_cat(
    variables=["ampdscode"], model="1", variant="-")
results.to_csv("results.csv")
results.to_pickle(os.path.join("pickle", "results.pickle"))
results = pd.read_pickle(
    os.path.join("pickle", "results.pickle"))

In [ ]:
def save_results(next_mod, results):
    results = results.append(next_mod)
    results.to_csv("results.csv")
    results.to_pickle(os.path.join("pickle", "results.pickle"))

In [ ]:
results = pd.read_pickle(
    os.path.join("pickle", "results.pickle"))
mod_2 = modelrun_text(
    text="prob_desc", model="2", variant="-", svd=None, ngram_range=(1, 1))

save_results(mod_2, results)

In [ ]:
results = pd.read_pickle(
    os.path.join("pickle", "results.pickle"))
mod_3 = modelrun_text(text="prob_desc", model="3",
                   variant="ngram_range=(1,2)", svd=None, ngram_range=(1, 2))
save_results(mod_3, results)

In [ ]:
results = pd.read_pickle(
    os.path.join("pickle", "results.pickle"))
mod_4 = modelrun_text(text="prob_desc", model="4",
                   variant="ngram_range=(1,3)", svd=None, ngram_range=(1, 3))
save_results(mod_4, results)

In [ ]:
results = pd.read_pickle(
    os.path.join("pickle", "results.pickle"))
mod_5 = modelrun_text(text="prob_desc_lemmat", model="5",
                   variant="lemmatized text", svd=None, ngram_range=(1, 1))
save_results(mod_5, results)

In [ ]:
results = pd.read_pickle(
    os.path.join("pickle", "results.pickle"))
mod_6 = modelrun_text(text="prob_desc_no_punc", model="6",
                   variant="no punctuation", svd=None, ngram_range=(1, 1))
save_results(mod_6, results)

In [ ]:
results = pd.read_pickle(
    os.path.join("pickle", "results.pickle"))
mod_7 = modelrun_text_tfidf(text="prob_desc", model="7",
                         variant="TF-IDF", svd=None, ngram_range=(1, 1))
save_results(mod_7, results)

In [ ]:
results = pd.read_pickle(
    os.path.join("pickle", "results.pickle"))
mod_8 = modelrun_all_count(text="prob_desc", others=[
                                   "ampdscode"], model="8", variant="-")
save_results(mod_8, results)

In [ ]:
# Function to recalculate DeLong CIs but with Bonferroni correction
def adjust_ci(x):    
    auc, auc_cov = delong_roc_variance(
        amb_test[y_varb].values,
        x[0][:, 1])
    
    alpha = 1 - (0.05/len(results))

    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

    ci = stats.norm.ppf(lower_upper_q, loc=auc, scale=auc_std)
    ci[ci > 1] = 1

    return f" {x[1]} ({ci[0].round(3)} - {ci[1].round(3)})"

In [ ]:
# Create new column that has both the ROC-AUC point estimate and predicted probabilities
results['adjust_roc'] = results.apply(
    lambda row: [row["Test y_pred_proba"],row["Test roc auc"]], axis = 1) 
results["Test roc auc (CI)"] = results['adjust_roc'].apply(lambda x: adjust_ci(x))

In [ ]:
results = pd.read_pickle(
    os.path.join("pickle", "results.pickle"))

## Results

### Tables

In [ ]:
crossval_tables = ["CV sensitivity", "CV specificity",
                   "CV precision", "CV true negative rate", "CV roc auc"]

# Shorter names
crossval_columns = ["Sensitivity", "Specificity",
                   "Precision", "TN rate", "ROC-AUC"]

test_tables = ["Test sensitivity", "Test specificity", "Test precision",
               "Test true negative rate", "Test roc auc (CI)"]

# Shorter names
test_columns = ["Sensitivity", "Specificity",
                   "Precision", "TN rate", "ROC-AUC (CI)"]

In [ ]:
# Cross validation
results_cv = results[crossval_tables]
results_cv.columns = crossval_columns
results_cv = pd.concat([results_cv], keys=["CV"], names = "", axis=1)

# Testing
results_test = results[test_tables]
results_test.columns = test_columns
results_test = pd.concat([results_test], keys=["Testing"], names = "", axis=1)

results_final  = pd.concat([results_cv, results_test], axis=1)

In [ ]:
# Results table for report
results_final.to_pickle(
    os.path.join("pickle", "results_final.pickle"))


In [ ]:
results_final.to_csv("results_final.csv")

### Calibration plot

In [ ]:
import scikitplot as skplt

In [ ]:
# Predicted probabilities
probas_list = [results["Test y_pred_proba"][1][:,1],
               results["Test y_pred_proba"][2][:,1],
               results["Test y_pred_proba"][4][:,1],
               results["Test y_pred_proba"][5][:,1], 
               results["Test y_pred_proba"][22][:,1],
               results["Test y_pred_proba"][23][:,1]]

# Model/classifier names
clf_names = [results.reset_index()["Model"][1],
             results.reset_index()["Model"][2],
             results.reset_index()["Model"][4],
             results.reset_index()["Model"][5],
             results.reset_index()["Model"][22],
             results.reset_index()["Model"][23]]

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111)

# Calibration curve


skplt.metrics.plot_calibration_curve(
    amb_test[y_varb].values, probas_list, clf_names, ax=ax)
fig.savefig("figures_new/calib_plot.jpg")

Taken from:
https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/

### ROC plot

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
fig = plt.figure(figsize=(12, 8))

# Generate a no skill prediction (majority class)
sim = [0 for _ in range(len(amb_test))]

sim_fpr, sim_tpr, _ = roc_curve(amb_test[y_varb].values, sim)

# FPRs and TPRs for each model
GBM1_fpr, GBM1_tpr, _ = roc_curve(amb_test[y_varb].values, 
                                  results["Test y_pred_proba"][1][:,1])
RF_bal1_fpr, RF_bal1_tpr, _  = roc_curve(amb_test[y_varb].values, 
                                         results["Test y_pred_proba"][2][:,1])
GBM2_fpr, GBM2_tpr, _  = roc_curve(amb_test[y_varb].values, 
                                         results["Test y_pred_proba"][3][:,1])

RF_bal2_fpr, RF_bal2_tpr, _  = roc_curve(amb_test[y_varb].values, 
                                         results["Test y_pred_proba"][4][:,1])

GBM8_fpr, GBM8_tpr, _  = roc_curve(amb_test[y_varb].values, 
                                         results["Test y_pred_proba"][22][:,1])

RF_bal8_fpr, RF_bal8_tpr, _  = roc_curve(amb_test[y_varb].values, 
                                         results["Test y_pred_proba"][23][:,1])

# Plot the roc curve for each model
plt.plot(GBM1_fpr, GBM1_tpr, label='1GBM',color="black", linewidth=2.5)
plt.plot(RF_bal1_fpr, RF_bal1_tpr, label='1RF_bal', color="blue", linewidth=2.5)
plt.plot(GBM2_fpr, GBM2_tpr, label='2GBM', color="deepskyblue", linewidth=2.5)
plt.plot(RF_bal2_fpr, RF_bal2_tpr, label='2RF_bal', color="green", linewidth=2.5)
plt.plot(GBM8_fpr, GBM8_tpr, label='8GBM', color="greenyellow", linewidth=2.5)
plt.plot(RF_bal8_fpr, RF_bal8_tpr, label='8RF_bal', color="red", linewidth=2.5)
plt.plot(sim_fpr, sim_tpr, linestyle='--', label='', color="black", linewidth=2.5)

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.legend(loc="lower right", markerscale=200)

plt.savefig("figures_new/roc_auc.jpg")

### Feature importances

In [ ]:
fig = plt.figure(figsize=(10, 10))

ax1 = fig.add_subplot(221)

plt.subplots_adjust(wspace=0.5, hspace=0.5)

# Get features from results tables and sort in descending order or importance
pd.Series(results["Feature Imps"][4]).sort_values(
    ascending=True).plot.barh(color="green",ax=ax1)
ax1.set_xlabel("Importance")
ax1.set_ylabel("Feature")
ax1.title.set_text('Model 2GBM')


ax2 = fig.add_subplot(222, xmargin=3)
pd.Series(results["Feature Imps"][5]).sort_values(
    ascending=True).plot.barh(color="green",ax=ax2)
ax2.set_xlabel("Importance")
ax2.set_ylabel("Feature")
ax2.title.set_text('Model 2RF_bal')

ax1 = fig.add_subplot(223, ymargin=3)
# Get features from results tables and sort in descending order or importance
pd.Series(results["Feature Imps"][22]).sort_values(
    ascending=True)[10:].plot.barh(color="green",ax=ax1)
ax1.set_xlabel("Importance")
ax1.set_ylabel("Feature")
ax1.title.set_text('Model 8GBM')


ax2 = fig.add_subplot(224)
pd.Series(results["Feature Imps"][23]).sort_values(
    ascending=True)[10:].plot.barh(color="green",ax=ax2)
ax2.set_xlabel("Importance")
ax2.set_ylabel("Feature")
ax2.title.set_text('Model 8RF_bal')
fig.savefig("figures_new/imp_text.jpg")


### Conveyance rate and occurance

In [ ]:
pre_made =["(ALL INCS)"]

# Chart showing frequency and proportion conveyed by term ("Skinny fries" chart
sns.set_style("dark")
amb_train["(ALL INCS)"] = 1
fig = plt.figure(figsize=(12, 8))
vabs = ["heart","hx", "head", "vomiting", "dob","and", "not", "in", "(ALL INCS)"
        , "pat", "?", "uncons", "caller", "lying", "nfda", "homeless"]

# Fix spacing
ind = np.arange(len(vabs))*0.5
#ind[6:] = ind[6:]+0.5
ind[7:] = ind[7:]+0.5
ind[8:] = ind[8:]+0.5
ind[9:] = ind[9:]+0.5

conv = [] # conveyance rate
width = [] # bar width

# Iterate through all of the terms to get conveyance rate and frequency of occurrence
for i in range(len(vabs)):
    if vabs[i] not in pre_made:
        my_regex = r"(" + re.escape(vabs[i]) + r")"
        amb_train[vabs[i]] = amb_train.prob_desc.apply(
            lambda x: 1 if re.search(my_regex, x) else 0)

    conv.append(len(amb_train.loc[(amb_train[vabs[i]] == 1) & (
        amb_train.conveyed == 1)])*100/len(amb_train[amb_train[vabs[i]] == 1]))

    width.append(amb_train[vabs[i]].sum()/len(amb_train))

fig = plt.figure(figsize=(13, 7))
p1 = plt.bar(ind, conv, width=width, edgecolor="green", color="green")
plt.xticks(ind, vabs)
plt.ylabel("% conveyed")
plt.xlabel("Term (bar width proportional to frequency)")

plt.savefig("figures_new/skinny.jpg")

In [ ]:
# Compare string lengths with "nfda" in problem description
amb_train[amb_train.nfda ==1]["string_len"].mean()
amb_train["string_len"].mean()

### Summary stats for train and  test sets

In [ ]:
# Function for displaying cross validation and test set results side by side
def desc_stats2(vab):    
    a = desc_stats1(vab, amb_train, "Training set")
    b = desc_stats1(vab, amb_test, "Test set")
      
    z  = pd.concat([a, b], axis=1)
    
    return z

In [ ]:
# MPDS summary table for report
mpds_table = desc_stats2("ampds_full")

# Adding an "all incidents" row
amb_train["dummy"] = 1
amb_test["dummy"] = 1
all_incs = desc_stats2("dummy")
all_incs = all_incs.rename(index={1: 'All incidents'})
full_tab = all_incs.append(mpds_table)
full_tab.to_csv("desc_stats.csv")

In [ ]:
# Text summary stats (as earlier but for test set too)
test_text = {"Average string length": 
                amb_test["string_len"].mean().round(1),          
                "Average string length (not conveyed)":
                amb_test.loc[amb_test.conveyed ==0 ]["string_len"].mean().round(1),
           "ave_size_unique": av_length(amb_test)}


pd.DataFrame({"Training set":training_text,"Test set":test_text})

## Package version numbers

In [ ]:
packages = ["numpy", "pandas", "matplotlib", "seaborn", "nltk",
            "spacy", "sklearn", "wordcloud", "scipy", "scikitplot"]

versions = [np.__version__, pd.__version__, matplotlib.__version__,
            sns.__version__, nltk.__version__, spacy.__version__,
            sklearn.__version__, "1.5.0", scipy.__version__, skplt.__version__]


print(pd.DataFrame.from_dict({"Package": packages,
                        "Version": versions}
                      ).set_index("Package"))

          Version
Package           
numpy       1.15.4
pandas      0.23.4
matplotlib   3.0.2
seaborn      0.9.0
nltk         3.4.1
spacy        2.1.3
sklearn     0.20.2
wordcloud    1.5.0
scipy        1.1.0
scikitplot   0.3.7